电商18 180412126 王佳琦

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])#数据集加载时，默认的图片格式是 numpy，所以通过 transforms 转换成 Tensor。
                                                              #然后，再对输入图片进行标准化。
    #前面的（0.5，0.5，0.5） 是 R G B 三个通道上的均值， 后面(0.5, 0.5, 0.5)是三个通道的标准差，Normalize对每个通道执行以下操作：image =（图像-平均值）/ std

batch_size = 4

#batchsize: 批大小 num_works:num_works: 是否多进程读取数据 shuffle: 每个 epoch 是否乱序 drop_last: 当样本数不能被 batchsize 整除时，是否舍弃最后一批数据

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [34]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)             #定义卷积层和池化层
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)       #定义全连接层
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()

In [35]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()         #用交叉熵作loss function
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)  #随机梯度下降法

In [36]:
from visdom import Visdom
visdom_show = Visdom(env="trainloss")
visdom_accu = Visdom(env="testaccu")
visdom_test = Visdom(env="testloss")
for epoch in range(50):  # loop over the dataset multiple times
        #可视化

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data          

        # zero the parameter gradients
        optimizer.zero_grad()       #梯度从零开始

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
      

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            if (i+1)==12000:
                visdom_show.line(
                    X=[epoch+1],
                    Y=[float(running_loss / 2000)],
                    win='accu and loss',
                    name='train_loss',
                    opts=dict(title='accu and loss',  legend=['train_loss']),
                    update='append')
            running_loss = 0.0
    correct = 0
    total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
    with torch.no_grad():
        running_loss2=0
        for data in testloader:
            images, labels = data
            # calculate outputs by running images through the network
            outputs = net(images)        #放进模型里
            # the class with the highest energy is what we choose as prediction
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()  #相同的累加
            loss = criterion(outputs, labels)
            running_loss2 += loss.item()
        print('Accuracy of the network on the 2500 test images: %d %%' % (100 * correct / total))
        visdom_accu.line(
            X=[epoch+1],
            Y=[(correct / total)],
            win='accu and loss',
            opts=dict(title='accu and loss',  legend=['test_accu']),
            name='test_accu',
            update='append')
        print('[%d] loss: %.3f' %
                  (epoch + 1,running_loss2 / 2500))
        visdom_test.line(
            X=[epoch+1],
            Y=[float(running_loss2 / 2500)],
            win='accu and loss',
            opts=dict(title='accu and loss',  legend=['test_loss']),
            name='test_loss',
            update='append')
        print(i)
print('Finished Training')

Setting up a new session...
Setting up a new session...
Setting up a new session...


[1,  2000] loss: 2.240
[1,  4000] loss: 1.920
[1,  6000] loss: 1.697
[1,  8000] loss: 1.623
[1, 10000] loss: 1.546
[1, 12000] loss: 1.465
Accuracy of the network on the 2500 test images: 44 %
[1] loss: 1.511
12499
[2,  2000] loss: 1.421
[2,  4000] loss: 1.363
[2,  6000] loss: 1.332
[2,  8000] loss: 1.320
[2, 10000] loss: 1.283
[2, 12000] loss: 1.270
Accuracy of the network on the 2500 test images: 54 %
[2] loss: 1.320
12499
[3,  2000] loss: 1.211
[3,  4000] loss: 1.190
[3,  6000] loss: 1.182
[3,  8000] loss: 1.170
[3, 10000] loss: 1.172
[3, 12000] loss: 1.159
Accuracy of the network on the 2500 test images: 58 %
[3] loss: 1.192
12499
[4,  2000] loss: 1.085
[4,  4000] loss: 1.080
[4,  6000] loss: 1.101
[4,  8000] loss: 1.080
[4, 10000] loss: 1.097
[4, 12000] loss: 1.067
Accuracy of the network on the 2500 test images: 60 %
[4] loss: 1.132
12499
[5,  2000] loss: 0.993
[5,  4000] loss: 1.006
[5,  6000] loss: 1.018
[5,  8000] loss: 1.000
[5, 10000] loss: 1.020
[5, 12000] loss: 1.013
Accura

[38, 10000] loss: 0.616
[38, 12000] loss: 0.656
Accuracy of the network on the 2500 test images: 57 %
[38] loss: 1.711
12499
[39,  2000] loss: 0.520
[39,  4000] loss: 0.547
[39,  6000] loss: 0.586
[39,  8000] loss: 0.616
[39, 10000] loss: 0.589
[39, 12000] loss: 0.623
Accuracy of the network on the 2500 test images: 60 %
[39] loss: 1.730
12499
[40,  2000] loss: 0.473
[40,  4000] loss: 0.514
[40,  6000] loss: 0.578
[40,  8000] loss: 0.602
[40, 10000] loss: 0.602
[40, 12000] loss: 0.594
Accuracy of the network on the 2500 test images: 60 %
[40] loss: 1.736
12499
[41,  2000] loss: 0.495
[41,  4000] loss: 0.521
[41,  6000] loss: 0.545
[41,  8000] loss: 0.614
[41, 10000] loss: 0.596
[41, 12000] loss: 0.625
Accuracy of the network on the 2500 test images: 60 %
[41] loss: 1.639
12499
[42,  2000] loss: 0.494
[42,  4000] loss: 0.538
[42,  6000] loss: 0.590
[42,  8000] loss: 0.614
[42, 10000] loss: 0.599
[42, 12000] loss: 0.609
Accuracy of the network on the 2500 test images: 59 %
[42] loss: 1.7

In [30]:
outputs = net(images)

In [31]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in testloader:
        images, labels = data
        # calculate outputs by running images through the network
        outputs = net(images)        #放进模型里
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()  #相同的累加

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 57 %


In [32]:
# prepare to count predictions for each class
correct_pred = {classname: 0 for classname in classes}  #定义列表
total_pred = {classname: 0 for classname in classes}

# again no gradients needed
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predictions = torch.max(outputs, 1)
        # collect the correct predictions for each class
        for label, prediction in zip(labels, predictions):
            if label == prediction:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1


# print accuracy for each class
for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print("Accuracy for class {:5s} is: {:.1f} %".format(classname,
                                                   accuracy))

Accuracy for class plane is: 54.3 %
Accuracy for class car   is: 76.1 %
Accuracy for class bird  is: 43.7 %
Accuracy for class cat   is: 39.0 %
Accuracy for class deer  is: 39.7 %
Accuracy for class dog   is: 46.4 %
Accuracy for class frog  is: 72.7 %
Accuracy for class horse is: 61.3 %
Accuracy for class ship  is: 76.7 %
Accuracy for class truck is: 65.9 %


[WinError 10054] 远程主机强迫关闭了一个现有的连接。
[WinError 10054] 远程主机强迫关闭了一个现有的连接。
[WinError 10054] 远程主机强迫关闭了一个现有的连接。
[WinError 10054] 远程主机强迫关闭了一个现有的连接。
[WinError 10054] 远程主机强迫关闭了一个现有的连接。
[WinError 10054] 远程主机强迫关闭了一个现有的连接。
[WinError 10054] 远程主机强迫关闭了一个现有的连接。
[WinError 10054] 远程主机强迫关闭了一个现有的连接。
[WinError 10054] 远程主机强迫关闭了一个现有的连接。
[WinError 10054] 远程主机强迫关闭了一个现有的连接。
[WinError 10054] 远程主机强迫关闭了一个现有的连接。
[WinError 10054] 远程主机强迫关闭了一个现有的连接。
[WinError 10054] 远程主机强迫关闭了一个现有的连接。
[WinError 10061] 由于目标计算机积极拒绝，无法连接。
[WinError 10061] 由于目标计算机积极拒绝，无法连接。
[WinError 10061] 由于目标计算机积极拒绝，无法连接。
[WinError 10061] 由于目标计算机积极拒绝，无法连接。
[WinError 10061] 由于目标计算机积极拒绝，无法连接。
[WinError 10061] 由于目标计算机积极拒绝，无法连接。
[WinError 10061] 由于目标计算机积极拒绝，无法连接。
[WinError 10061] 由于目标计算机积极拒绝，无法连接。
[WinError 10061] 由于目标计算机积极拒绝，无法连接。
[WinError 10061] 由于目标计算机积极拒绝，无法连接。
[WinError 10061] 由于目标计算机积极拒绝，无法连接。
[WinError 10061] 由于目标计算机积极拒绝，无法连接。
[WinError 10061] 由于目标计算机积极拒绝，无法连接。
[WinError 10061] 由于目标计算机积极拒绝，无法连接。
[WinError 10061] 由于目标计算机积极拒绝，无法连接。
[WinError 10061] 由于目